In [3]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [4]:
dic_table= pickle.load(open("dic_table",'rb'))
df_historical_data= pd.read_csv('clean_fifa_worldcup_historical_data.csv')
df_fixture= pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [5]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
859,Russia,Croatia,2018,2,2,4
860,France,Belgium,2018,1,0,1
861,Croatia,England,2018,2,1,3
862,Belgium,England,2018,2,0,2


In [6]:
#dividir df en df_home and df_away
df_home= df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away= df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

	

In [7]:
#renombrando columnas
df_home=df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [8]:
#Concatenar df_home and df_away
df_team_strength=pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.717949,1.166667
Australia,0.812500,1.937500
Austria,1.576923,1.692308
...,...,...
Uruguay,1.603774,1.339623
Wales,0.800000,0.800000
West Germany,2.050847,1.254237


In [9]:
def predict_points(home,away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #goals_scored * goals_conceded
        lamb_home= df_team_strength.at[home,'GoalsScored']* df_team_strength.at[away,'GoalsConceded']
        lamb_away= df_team_strength.at[away,'GoalsScored']* df_team_strength.at[home,'GoalsConceded']
        prob_home,prob_away,prob_draw= 0,0,0
        for x in range(0,11):#number of goals home team
            for y in range(0,11):#number of goals away team
                p= poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw += p
                elif x>y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home= 3 * prob_home + prob_draw
        points_away= 3 * prob_away + prob_draw
        return (points_home,points_away)
    else:
        return(0,0)

In [10]:
#Testeo de partidos: Argentina vs Mexico - England vs United States - Qatar vs Ecuador
predict_points('Argentina','Mexico')
predict_points('England','United States')
predict_points('Qatar','Ecuador')

(0, 0)

In [11]:
#Prediccion Mundial
df_fixture_group_48= df_fixture[:48].copy()
df_fixture_knockout= df_fixture[48:56].copy()
df_fixture_quarter= df_fixture[56:60].copy()
df_fixture_semi= df_fixture[60:62].copy()
df_fixture_final= df_fixture[62:].copy()

In [12]:
#Corriendo todos los partidos de la fase de grupo y actualizar las tablas 
for group in dic_table:
    teams_in_group= dic_table[group]['Team'].values
    df_fixture_group_6= df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away= row['home'], row['away']
        points_home, points_away= predict_points(home,away)
        dic_table[group].loc[dic_table[group]['Team'] == home, 'Pts']+= points_home
        dic_table[group].loc[dic_table[group]['Team'] == away, 'Pts']+=points_away
        
    dic_table[group]= dic_table[group].sort_values('Pts',ascending= False).reset_index()
    dic_table[group]= dic_table[group][['Team','Pts']]
    dic_table[group]= dic_table[group].round(0)

/var/folders/mv/9jtv_q3s0gs6ndhpn_7nsv900000gq/T/ipykernel_676/152537164.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.87343108]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dic_table[group].loc[dic_table[group]['Team'] == home, 'Pts']+= points_home
/var/folders/mv/9jtv_q3s0gs6ndhpn_7nsv900000gq/T/ipykernel_676/152537164.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39716107]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dic_table[group].loc[dic_table[group]['Team'] == home, 'Pts']+= points_home
/var/folders/mv/9jtv_q3s0gs6ndhpn_7nsv900000gq/T/ipykernel_676/152537164.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.78567341]' has 

In [13]:
#tablas actualizadas
dic_table['Grupo A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


In [14]:
#Octavos: df_fixture_knockout
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [15]:
# Diccionario para traducir grupos al formato inglés
traductor_grupos = {
    'Grupo A': 'Group A',
    'Grupo B': 'Group B',
    'Grupo C': 'Group C',
    'Grupo D': 'Group D',
    'Grupo E': 'Group E',
    'Grupo F': 'Group F',
    'Grupo G': 'Group G',
    'Grupo H': 'Group H'
}

# Iterar sobre los grupos y actualizar el fixture
for grupo in dic_table:
    # Traducir el grupo al formato esperado en df_fixture_knockout
    group_english = traductor_grupos[grupo]
    
    # Ganadores y subcampeones del grupo
    group_winner = dic_table[grupo].loc[0, 'Team']
    runners_up = dic_table[grupo].loc[1, 'Team']
    
    # Reemplazar en el fixture
    df_fixture_knockout.replace({
        f'Winners {group_english}': group_winner,
        f'Runners-up {group_english}': runners_up
    }, inplace=True)

# Agregar la columna "Winner"
df_fixture_knockout['Winner'] = '?'

# Verifica el resultado
print(df_fixture_knockout)


           home     score         away  year Winner
48  Netherlands  Match 49        Wales  2022      ?
49    Argentina  Match 50      Denmark  2022      ?
50       France  Match 52       Poland  2022      ?
51      England  Match 51      Senegal  2022      ?
52      Germany  Match 53      Belgium  2022      ?
53       Brazil  Match 54      Uruguay  2022      ?
54      Croatia  Match 55        Spain  2022      ?
55     Portugal  Match 56  Switzerland  2022      ?


In [16]:
def get_winner (df_fixture_update):
    for index, row in df_fixture_update.iterrows():
        home,away =row['home'],row['away' ]
        points_home,points_away = predict_points(home,away)
        if points_home > points_away :
            Winner= home
        else:
            Winner=away 
        df_fixture_update.loc[index, 'Winner']= Winner
    return df_fixture_update

In [29]:
df_fixture_knockout= get_winner(df_fixture_knockout)
df_fixture_knockout

,home,score,away,year,Winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Croatia
55,Portugal,Match 56,Switzerland,2022,Portugal


In [25]:
print(df_fixture_knockout[['score', 'Winner']])


       score       Winner
48  Match 49  Netherlands
49  Match 50    Argentina
50  Match 52       France
51  Match 51      England
52  Match 53      Germany
53  Match 54       Brazil
54  Match 55      Croatia
55  Match 56     Portugal


In [33]:
#create update_table function
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = row['Winner']  # Ganador del partido anterior
        match = row['score']    # Número de partido del fixture anterior

        # Reemplazar en 'home' y 'away' en el siguiente fixture
        df_fixture_round_2['home'] = df_fixture_round_2['home'].replace(f'Winners {match}', winner)
        df_fixture_round_2['away'] = df_fixture_round_2['away'].replace(f'Winners {match}', winner)

    # Mantener 'Winner' de la siguiente ronda con valor '?'
    df_fixture_round_2['Winner'] = '?'
    return df_fixture_round_2


In [32]:
print(df_fixture_knockout[['score', 'Winner']])
print(df_fixture_quarter[['home', 'away']])


       score       Winner
48  Match 49  Netherlands
49  Match 50    Argentina
50  Match 52       France
51  Match 51      England
52  Match 53      Germany
53  Match 54       Brazil
54  Match 55      Croatia
55  Match 56     Portugal
                home              away
56  Winners Match 53  Winners Match 54
57  Winners Match 49  Winners Match 50
58  Winners Match 55  Winners Match 56
59  Winners Match 51  Winners Match 52


In [34]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,Winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Croatia,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [35]:
get_winner(df_fixture_quarter)

,home,score,away,year,Winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Croatia,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [36]:
update_table(df_fixture_quarter,df_fixture_semi)

,home,score,away,year,Winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [37]:
get_winner(df_fixture_semi)

,home,score,away,year,Winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [38]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [39]:
get_winner(df_fixture_final)

,home,score,away,year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
